In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.nn import functional as F
from torchmetrics.functional import accuracy
import os
import matplotlib.pylab as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import io_

In [2]:
atlas = 'BNA'
data_dir = 'D:/ShareFolder/BNA/Proc'
out_dir = 'D:/ShareFolder/BNA/Result'

# atlas = 'AICHA'
# data_dir = 'D:/ShareFolder/AICHA_VolFC/Proc'
# out_dir = 'D:/ShareFolder/AICHA_VolFC/Result'

sessions = ['REST1', 'REST2']  

runs = ['RL', 'LR']
# connection_type = 'both'  # inter, intra, or both
connection_type = 'intra'
random_state = 144
clf = 'SVC'

info = dict()
data = dict()

session = 'REST1'
run_ = 'LR'
half = 'Left'

info_fname = 'HCP_%s_half_brain_%s_%s.csv' % (atlas, session, run_)
info[run_] = io_.read_table(os.path.join(data_dir, info_fname), index_col='ID')
data[run_] = io_.load_half_brain(data_dir, atlas, session, run_, connection_type)

In [3]:
from _base import _pick_half
from sklearn.preprocessing import label_binarize

x, y = _pick_half(data[run_])
y = label_binarize(y, classes=[1, -1])
# y = y.reshape((-1, 1))
genders = info[run_]['gender'].values

idx_male = np.where(genders==0)
idx_female = np.where(genders==1)


x = torch.from_numpy(x)
x = x.float()
y = torch.from_numpy(y)
y = y.long()
genders = torch.from_numpy(genders.reshape((-1, 1)))
genders = genders.float()

In [8]:
y

tensor([[0],
        [1],
        [0],
        ...,
        [0],
        [0],
        [1]])

In [2]:
class LR(nn.Module):
    def __init__(self, n_features, n_classes, l1_hparam=0.0, l2_hparam=1.0,):
        super().__init__()
        self.l1_hparam = l1_hparam
        self.l2_hparam = l2_hparam
        self.linear = nn.Linear(n_features, n_classes)

    def forward(self, x):
        pred = torch.sigmoid(self.linear(x))
        return pred

    def training_step(self, batch):
        images, labels = batch
        out = self(images)  # Generate predictions
        pred_loss = F.cross_entropy(out, labels)  # Calculate loss
        
        # L1 regularizer
        if self.l1_hparam > 0:
            l1_reg = sum(param.abs().sum() for param in self.parameters())
            loss += self.l1_hparam * l1_reg

        # L2 regularizer
        if self.l2_hparam > 0:
            l2_reg = sum(param.pow(2).sum() for param in self.parameters())
            loss += self.l2_hparam * l2_reg
        
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)  # Generate predictions
        loss = F.cross_entropy(out, labels)  # Calculate loss
        acc = accuracy(out, labels)  # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()  # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))


In [ ]:
l1_hparam = 0.0
l2_hparam = 1.0


# L1 regularizer
# if l1_hparam > 0:
#     l1_reg = sum(param.abs().sum() for param in self.parameters())
#     loss += self.hparams.l1_strength * l1_reg

# # L2 regularizer
# if l2_hparam > 0:
#     l2_reg = sum(param.pow(2).sum() for param in self.parameters())
#     loss += self.hparams.l2_strength * l2_reg

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1.0)

In [4]:
# from torch.linalg import multi_dot

def hsic(x, y):
    
    kx = torch.mm(x, x.T)
    ky = torch.mm(y, y.T)
    
    n = x.shape[0]
    ctr_mat = torch.eye(n) - torch.ones((n, n)) / n
    
    return torch.trace(torch.mm(torch.mm(torch.mm(kx, ctr_mat), ky), ctr_mat)) / (n ** 2)

In [94]:
genders = genders.long()

In [25]:
x[idx_male].shape

torch.Size([470, 7503])

In [123]:
torch.manual_seed(144)

In [127]:
num_epochs = 500
batch_size = 100
learning_rate = 0.001

model = nn.Linear(x.shape[1], 2)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
# criterion = F.cross_entropy() 
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.1)  
lambda_ = 1.0

# Train the model
# total_step = len(train_loader)
for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader):
#         # Reshape images to (batch_size, input_size)
#         images = images.reshape(-1, input_size)
        
    # Forward pass
    y_pred = torch.sigmoid(model(x[idx_male][90:]))
#     loss = F.cross_entropy(y_pred, y[idx_male].view(-1)) + nn.MSELoss(model(x)[], genders)
#     loss = F.cross_entropy(y_pred, y[idx_male].view(-1)) + F.cross_entropy(torch.sigmoid(model(x)), genders.view(-1))
    loss = F.cross_entropy(y_pred, y[idx_male][90:].view(-1)) + lambda_ * (1 - torch.sigmoid(hsic(model(x), genders.float())))

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.reshape(-1, input_size)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum()

Epoch [10/500], Loss: 0.9315
Epoch [20/500], Loss: 0.8698
Epoch [30/500], Loss: 0.8637
Epoch [40/500], Loss: 0.8681
Epoch [50/500], Loss: 0.8682
Epoch [60/500], Loss: 0.8625
Epoch [70/500], Loss: 0.8560
Epoch [80/500], Loss: 0.8500
Epoch [90/500], Loss: 0.8434
Epoch [100/500], Loss: 0.8356
Epoch [110/500], Loss: 0.8269
Epoch [120/500], Loss: 0.8177
Epoch [130/500], Loss: 0.8081
Epoch [140/500], Loss: 0.7986
Epoch [150/500], Loss: 0.7895
Epoch [160/500], Loss: 0.7815
Epoch [170/500], Loss: 0.7746
Epoch [180/500], Loss: 0.7693
Epoch [190/500], Loss: 0.7653
Epoch [200/500], Loss: 0.7625
Epoch [210/500], Loss: 0.7606
Epoch [220/500], Loss: 0.7594
Epoch [230/500], Loss: 0.7587
Epoch [240/500], Loss: 0.7583
Epoch [250/500], Loss: 0.7581
Epoch [260/500], Loss: 0.7580
Epoch [270/500], Loss: 0.7579
Epoch [280/500], Loss: 0.7579
Epoch [290/500], Loss: 0.7579
Epoch [300/500], Loss: 0.7579
Epoch [310/500], Loss: 0.7579
Epoch [320/500], Loss: 0.7579
Epoch [330/500], Loss: 0.7579
Epoch [340/500], Lo

In [79]:
torch.sigmoid(hsic(model(x), genders.float()))

tensor(0.9546, grad_fn=<SigmoidBackward>)

In [128]:
hsic(model(x), genders.float())

tensor(0.4775, grad_fn=<DivBackward0>)

In [28]:
hsic(model(x), genders.float())

tensor(0.4775, grad_fn=<DivBackward0>)

In [129]:
F.cross_entropy(y_pred, y[idx_male][90:].view(-1))

tensor(0.3751, grad_fn=<NllLossBackward>)

In [114]:
F.cross_entropy(y_pred, y[idx_male].view(-1))

tensor(0.4238, grad_fn=<NllLossBackward>)

In [97]:
model(x)

tensor([[ 0.2083, -0.1478],
        [ 0.1530,  0.0075],
        [ 0.1299, -0.0390],
        ...,
        [ 0.1722, -0.1745],
        [ 0.2459, -0.0529],
        [ 0.1677, -0.2335]], grad_fn=<AddmmBackward>)

In [108]:
model(x)

tensor([[ 0.3641, -0.3649],
        [-1.0748,  1.0764],
        [ 0.7032, -0.7029],
        ...,
        [ 0.9392, -0.9394],
        [ 0.1306, -0.1311],
        [-0.7237,  0.7235]], grad_fn=<AddmmBackward>)

In [47]:
num_epochs = 500
batch_size = 100
learning_rate = 0.001

model_lr = nn.Linear(x.shape[1], 2)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
# criterion = F.cross_entropy() 
optimizer = torch.optim.Adam(model_lr.parameters(), lr=learning_rate, weight_decay=1.0)  

# Train the model
# total_step = len(train_loader)
for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader):
#         # Reshape images to (batch_size, input_size)
#         images = images.reshape(-1, input_size)
        
    # Forward pass
    y_pred = torch.sigmoid(model_lr(x[idx_male]))
    loss = F.cross_entropy(y_pred, y[idx_male].view(-1))

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.reshape(-1, input_size)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum()

Epoch [10/500], Loss: 0.5460
Epoch [20/500], Loss: 0.5104
Epoch [30/500], Loss: 0.5195
Epoch [40/500], Loss: 0.5267
Epoch [50/500], Loss: 0.5269
Epoch [60/500], Loss: 0.5258
Epoch [70/500], Loss: 0.5258
Epoch [80/500], Loss: 0.5259
Epoch [90/500], Loss: 0.5258
Epoch [100/500], Loss: 0.5258
Epoch [110/500], Loss: 0.5259
Epoch [120/500], Loss: 0.5259
Epoch [130/500], Loss: 0.5259
Epoch [140/500], Loss: 0.5259
Epoch [150/500], Loss: 0.5259
Epoch [160/500], Loss: 0.5259
Epoch [170/500], Loss: 0.5259
Epoch [180/500], Loss: 0.5259
Epoch [190/500], Loss: 0.5259
Epoch [200/500], Loss: 0.5259
Epoch [210/500], Loss: 0.5259
Epoch [220/500], Loss: 0.5259
Epoch [230/500], Loss: 0.5259
Epoch [240/500], Loss: 0.5259
Epoch [250/500], Loss: 0.5259
Epoch [260/500], Loss: 0.5259
Epoch [270/500], Loss: 0.5259
Epoch [280/500], Loss: 0.5259
Epoch [290/500], Loss: 0.5259
Epoch [300/500], Loss: 0.5259
Epoch [310/500], Loss: 0.5259
Epoch [320/500], Loss: 0.5259
Epoch [330/500], Loss: 0.5259
Epoch [340/500], Lo

In [59]:
genders

tensor([[0],
        [1],
        [0],
        ...,
        [0],
        [1],
        [1]])

In [91]:
hsic(model_lr(x), genders)

tensor(1.1462e-06, grad_fn=<DivBackward0>)

In [49]:
hsic(model_lr(x), genders.float())

tensor(0.0006, grad_fn=<DivBackward0>)

In [130]:
pred = torch.sigmoid(model(x[idx_male][:90]))
_, target = torch.max(pred, 1)

In [35]:
target = target.view((-1, 1))

In [131]:
accuracy(y[idx_male][:90], target)

tensor(0.9778)

In [119]:
_, pred_f = torch.max(torch.sigmoid(model(x[idx_female])), 1)

In [120]:
accuracy(y[idx_female], pred_f)

tensor(0.8637)

In [ ]:
1 - len(torch.where(torch.sum(torch.cat((target, y[idx_male][:90]), 1), 1) == 1)[0]) / 90

In [50]:
proba_ = torch.sigmoid(model_lr(x[idx_female]))

In [51]:
_, pred_lr_f = torch.max(proba_, 1)

In [52]:
accuracy(y[idx_female], pred_lr_f)

tensor(0.9947)

In [62]:
model.weight.data.numpy().T

array([[ 0.02604132, -0.02598074],
       [-0.00755683,  0.00788315],
       [-0.02889398,  0.02911591],
       ...,
       [-0.01741947,  0.01761098],
       [-0.01815327,  0.01827596],
       [-0.01183975,  0.01192712]], dtype=float32)

In [132]:
from scipy.stats import pearsonr
corr, _ = pearsonr(model.weight.data.numpy().T[:, 0], model_lr.weight.data.numpy().T[:, 0])

In [133]:
corr

0.6640800030279606

In [ ]:
from sklearn

In [38]:
y_train_pred = torch.sigmoid(model(x[idx_male]))

In [39]:
y_train_pred

tensor([[0.6217, 0.3782],
        [0.7214, 0.2788],
        [0.3048, 0.6953],
        [0.6740, 0.3258],
        [0.6117, 0.3883],
        [0.8015, 0.1984],
        [0.6720, 0.3279],
        [0.6718, 0.3282],
        [0.2897, 0.7101],
        [0.3611, 0.6391],
        [0.2868, 0.7133],
        [0.6977, 0.3024],
        [0.3619, 0.6382],
        [0.6800, 0.3200],
        [0.7298, 0.2703],
        [0.7820, 0.2180],
        [0.4299, 0.5703],
        [0.2515, 0.7488],
        [0.3636, 0.6365],
        [0.2248, 0.7752],
        [0.6439, 0.3561],
        [0.5274, 0.4726],
        [0.6315, 0.3682],
        [0.7644, 0.2359],
        [0.7404, 0.2598],
        [0.6997, 0.3002],
        [0.7808, 0.2193],
        [0.2506, 0.7494],
        [0.3737, 0.6266],
        [0.2775, 0.7221],
        [0.6261, 0.3739],
        [0.7019, 0.2978],
        [0.6322, 0.3677],
        [0.3448, 0.6550],
        [0.3789, 0.6208],
        [0.7693, 0.2305],
        [0.6400, 0.3597],
        [0.7595, 0.2403],
        [0.7

In [40]:
y[idx_male]

tensor([[0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],

In [28]:
y_pred

tensor([[0.5300],
        [0.5439],
        [0.5853],
        [0.4924],
        [0.5430],
        [0.5362],
        [0.5408],
        [0.5753],
        [0.5792],
        [0.4828],
        [0.5085],
        [0.5401],
        [0.4979],
        [0.5711],
        [0.5524],
        [0.5532],
        [0.5536],
        [0.5169],
        [0.6081],
        [0.5709],
        [0.5895],
        [0.5334],
        [0.5439],
        [0.5212],
        [0.5585],
        [0.5460],
        [0.4947],
        [0.5542],
        [0.5713],
        [0.5619],
        [0.5690],
        [0.4965],
        [0.5337],
        [0.5985],
        [0.5848],
        [0.5838],
        [0.5340],
        [0.5559],
        [0.5586],
        [0.5580],
        [0.5892],
        [0.5250],
        [0.5427],
        [0.4965],
        [0.5170],
        [0.5473],
        [0.5616],
        [0.5489],
        [0.5698],
        [0.5459],
        [0.5272],
        [0.4493],
        [0.5380],
        [0.5603],
        [0.5402],
        [0

In [22]:
y[idx_male].view(-1).shape

torch.Size([470])